In [1]:
import tweepy 
import numpy as np 
import pandas as pd 
from datetime import datetime 
import matplotlib.pyplot as plt 
from matplotlib import style 
style.use('ggplot')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
analyzer=SentimentIntensityAnalyzer()

consumer_key= '7Pe8j6ayNyx0aCIFRZCKjfFZ2'
consumer_secret='LJtWZJtWsgb54ogSXIwgnEd8kHAdzYZvneRclbM8dGTujXuVrG'
access_token='2479773992-WyHyGnDVdzgg1wGByRi5XOBZmxmFzoHJHXito27'
access_token_secret='PtLYC4V5eqgPC18rSSN0QsqG6wTesFOnrSLSqiiO1t6aJ'

auth= tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API (auth, parser=tweepy.parsers.JSONParser())

In [2]:
target_terms=('@BBC','@CBS','@CNN','@Fox','@New York times')

min_tweets=5
max_tweets=10000
max_followers=2500
max_following=2500
lang='en'

counter=1
sentiments=[]

# List to hold results
results_list = []

# Loop through all target terms
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(
            target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate list
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)}

    # Print the Sentiments
    print(sentiment)
    print()
    
    # Append news results to 'results_list'
    results_list.append(sentiment)



{'User': '@BBC', 'Compound': -0.007042006269592478, 'Positive': 0.0811191222570533, 'Neutral': 0.08302507836990596, 'Negative': 0.8358652037617553, 'Tweet Count': 319}

{'User': '@CBS', 'Compound': 0.1302482993197279, 'Positive': 0.07579931972789114, 'Neutral': 0.03054081632653061, 'Negative': 0.8936496598639456, 'Tweet Count': 294}

{'User': '@CNN', 'Compound': 0.0256978947368421, 'Positive': 0.09726736842105263, 'Neutral': 0.07875578947368421, 'Negative': 0.8239431578947369, 'Tweet Count': 475}

{'User': '@Fox', 'Compound': -0.019988377192982465, 'Positive': 0.07717105263157895, 'Neutral': 0.07517105263157894, 'Negative': 0.8476578947368422, 'Tweet Count': 456}

{'User': '@New York times', 'Compound': -0.91995, 'Positive': 0.0, 'Neutral': 0.46399999999999997, 'Negative': 0.536, 'Tweet Count': 4}



In [3]:
sentiments_df=pd.DataFrame(results_list).set_index('User').round(3)
sentiments_df.head()


,Compound,Negative,Neutral,Positive,Tweet Count
User,,,,,
@BBC,-0.007,0.836,0.083,0.081,319
@CBS,0.130,0.894,0.031,0.076,294
@CNN,0.026,0.824,0.079,0.097,475
@Fox,-0.020,0.848,0.075,0.077,456
@New York times,-0.920,0.536,0.464,0.000,4


In [ ]:
sentiments_df.plot(kind="scatter", x="Tweets Ago", y="Tweet Polarity", grid=True, figsize=(-1,1),
              title="Sentiment Analysis of Media Tweet")
plt.scatter(x_axis,'User;, marker='O', facecolors='red','yellow','orange''black','bule' )
plt.show()